### Este código requer:

- Uma conta no Google Earth Engine (GEE);
- Sincronização da conta com o ambiente Python utilizado;
- Instalação dos frameworks ee e geemap, além de bibliotecas padrão, como:
    - pandas,
    - geopandas,
    - shapely.

In [1]:
import ee
import pandas as pd
import geopandas as gpd
import geemap
import requests
import pandas as pd
import zipfile
import io
from shapely.geometry import Point

In [ ]:
df = pd.DataFrame()

lista2 = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']


for i in lista2:
    url = f"https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/mensal/Brasil/focos_mensal_br_2023{i}.zip"
    response = requests.get(url)
    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            print("Arquivos contidos no ZIP:", z.namelist())
            with z.open(z.namelist()[0]) as f:
                df_temp = pd.read_csv(f)
                if not df_temp.empty:
                    df = pd.concat([df, df_temp], ignore_index=True)
                    print("DataFrame carregado com sucesso!")

    else:
        print(f"Erro ao acessar a URL: {response.status_code}")

for i in lista2:
    url = f"https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/mensal/Brasil/focos_mensal_br_2024{i}.csv"
    response = requests.get(url)
    
    if response.status_code == 200:
        df_temp = pd.read_csv(io.BytesIO(response.content))
        if not df_temp.empty:
            df = pd.concat([df, df_temp], ignore_index=True)
            print(f"DataFrame do mês {i} carregado com sucesso!")
    else:
        print(f"Erro ao acessar a URL para o mês {i}: {response.status_code}")

df['data_hora_gmt'] = pd.to_datetime(df['data_hora_gmt'], errors='coerce')

primeiros_4 = df['data_hora_gmt'].head(4)

df['Ano'] = df['data_hora_gmt'].dt.year

df.to_csv("Incendios_2023_2024.csv", index=False)

Arquivos contidos no ZIP: ['focos_mensal_br_202301.csv']
DataFrame carregado com sucesso!
Arquivos contidos no ZIP: ['focos_mensal_br_202302.csv']
DataFrame carregado com sucesso!
Arquivos contidos no ZIP: ['focos_mensal_br_202303.csv']
DataFrame carregado com sucesso!
Arquivos contidos no ZIP: ['focos_mensal_br_202304.csv']
DataFrame carregado com sucesso!
Arquivos contidos no ZIP: ['focos_mensal_br_202305.csv']
DataFrame carregado com sucesso!
Arquivos contidos no ZIP: ['focos_mensal_br_202306.csv']
DataFrame carregado com sucesso!
Arquivos contidos no ZIP: ['focos_mensal_br_202307.csv']
DataFrame carregado com sucesso!
Arquivos contidos no ZIP: ['focos_mensal_br_202308.csv']
DataFrame carregado com sucesso!
Arquivos contidos no ZIP: ['focos_mensal_br_202309.csv']
DataFrame carregado com sucesso!
Arquivos contidos no ZIP: ['focos_mensal_br_202310.csv']
DataFrame carregado com sucesso!
Arquivos contidos no ZIP: ['focos_mensal_br_202311.csv']
DataFrame carregado com sucesso!
Arquivos c

In [13]:
df = df.loc[df['municipio'].isin(["MARICÁ", "SAQUAREMA", "ARARUAMA", "RIO BONITO", "TANGUÁ"])]

df['municipio'].value_counts()

municipio
ARARUAMA      601
SAQUAREMA     404
MARICÁ        393
RIO BONITO    390
TANGUÁ        151
Name: count, dtype: int64

In [15]:
df['lat'] = df['lat'].astype('float32')
df['lon'] = df['lon'].astype('float32')

geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]

gdf = gpd.GeoDataFrame(df, geometry=geometry)

gdf.set_crs(epsg=4326, inplace=True)

,id,lat,lon,data_hora_gmt,satelite,municipio,estado,pais,municipio_id,estado_id,pais_id,numero_dias_sem_chuva,precipitacao,risco_fogo,bioma,frp,Ano,geometry
22846,129fbe5e-09a2-3163-9cc4-6a4ad1a9a90a,-22.889919,-42.387272,2023-01-17 16:52:00,NOAA-20,SAQUAREMA,RIO DE JANEIRO,Brasil,3305505,33,33,3.0,0.00,-999.00,Mata Atlântica,1.7,2023,POINT (-42.38727 -22.88992)
23682,a28015b3-be16-3330-8259-4b4916d2b2ee,-22.759951,-42.303349,2023-01-18 16:34:00,NOAA-20,ARARUAMA,RIO DE JANEIRO,Brasil,3300209,33,33,NaN,NaN,NaN,Mata Atlântica,9.5,2023,POINT (-42.30335 -22.75995)
23687,2f575814-1267-3cdb-b2a6-a6368ae4e73b,-22.741449,-42.393620,2023-01-18 16:34:00,NOAA-20,ARARUAMA,RIO DE JANEIRO,Brasil,3300209,33,33,NaN,NaN,NaN,Mata Atlântica,2.2,2023,POINT (-42.39362 -22.74145)
35348,f14ec86b-2a90-31e8-b55e-5fa08b041cf4,-22.823040,-42.353161,2023-01-28 16:45:00,NOAA-20,ARARUAMA,RIO DE JANEIRO,Brasil,3300209,33,33,2.0,3.35,0.24,Mata Atlântica,3.6,2023,POINT (-42.35316 -22.82304)
52738,f7be7b2d-149f-3192-a97c-627a57c0f59a,-22.735069,-42.720341,2023-02-10 16:56:00,AQUA_M-T,TANGUÁ,RIO DE JANEIRO,Brasil,3305752,33,33,0.0,16.49,-999.00,Mata Atlântica,8.5,2023,POINT (-42.72034 -22.73507)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11660106,7e645eb4-b5b3-3c99-9364-0588f669ff2b,-22.821140,-42.602909,2024-10-18 16:26:00,NOAA-20,SAQUAREMA,RIO DE JANEIRO,Brasil,3305505,33,33,6.0,0.18,0.59,Mata Atlântica,18.3,2024,POINT (-42.60291 -22.82114)
11660107,c317258e-5bfd-3168-a686-ab652d31199c,-22.824480,-42.602371,2024-10-18 16:26:00,NOAA-20,SAQUAREMA,RIO DE JANEIRO,Brasil,3305505,33,33,6.0,0.18,0.54,Mata Atlântica,3.4,2024,POINT (-42.60237 -22.82448)
11738325,ab2f30f3-0d53-3515-872a-8758ba6c782a,-22.626381,-42.207470,2024-10-23 00:41:00,TERRA_M-M,ARARUAMA,RIO DE JANEIRO,Brasil,3300209,33,33,2.0,0.00,0.03,Mata Atlântica,8.0,2024,POINT (-42.20747 -22.62638)
11738326,d40f3ed1-f0f6-3d03-8ec3-b086fff42b6a,-22.627720,-42.217018,2024-10-23 00:41:00,TERRA_M-M,ARARUAMA,RIO DE JANEIRO,Brasil,3300209,33,33,2.0,0.00,0.03,Mata Atlântica,9.9,2024,POINT (-42.21702 -22.62772)


In [2]:
m = geemap.Map(center=[-22.9, -46.6], zoom=4)

#m.add_basemap('SATELLITE')

#m.add_gdf(gdf,layer_name="Incêndios 2023 - 2024")

#heat_data = [(point.y, point.x) for point in gdf.geometry]

#m.add_heatmap(heat_data, radius=15, opacity=0.95)

m

Map(center=[-22.9, -46.6], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [10]:
roi = m.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-98.1738, 18.0623, -76.8164, 31.6534)
start_date = "2024-10-09T12:00:00"
end_date = "2024-10-09T22:00:00"
data = "GOES-16"
scan = "full_disk"

timelapse = geemap.goes_timelapse(
    roi, "Hurricane_Milton.gif", start_date, end_date, data, scan, framesPerSecond=5, overlay_data="us_states"
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\Diego\OneDrive - Universidade de Vassouras\zzGitHub\Palestra-ENIC-XXI\Hurricane_Milton.gif


Output()

In [11]:
roi = m.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-50.0, -24.0, -35.0, -20.0)

start_date = "2024-10-24T12:00:00" # O erro estava na data, por isso estava gerando uma variavel None
end_date = "2024-10-24T22:00:00"
data = "GOES-16"
scan = "full_disk"

timelapse = geemap.goes_timelapse(
    roi, "Sudeste_Brasil.gif", start_date=start_date, end_date=end_date, data=data, scan=scan, framesPerSecond=5
)

geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\Diego\OneDrive - Universidade de Vassouras\zzGitHub\Palestra-ENIC-XXI\Sudeste_Brasil.gif


Output()